# Data Extraction ⬇️

In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "browser"

In [98]:
original_dataset = pd.read_csv('base_telecomx.csv')
original_dataset.head()

,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,account.Contract,account.PaymentMethod,account.Charges.Total,daily_bill
0,0,0,0,1,1,9.0,One year,Mailed check,593.30,2.20
1,0,1,0,0,0,9.0,Month-to-month,Mailed check,542.40,2.01
2,1,1,0,0,0,4.0,Month-to-month,Electronic check,280.85,2.34
3,1,1,1,1,0,13.0,Month-to-month,Electronic check,1237.85,3.17
4,1,0,1,1,0,3.0,Month-to-month,Mailed check,267.40,2.97


# Data Transformation 🔧

In [99]:
x = original_dataset.drop('Churn', axis=1)
y = original_dataset['Churn']

colunas = x.columns

## Data encoding

In [100]:
dataset_onehot = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary'), ['account.Contract', 'account.PaymentMethod']),
    remainder = 'passthrough',
    sparse_threshold = 0
)

x = dataset_onehot.fit_transform(x)
dataset_onehot.get_feature_names_out(colunas)

df_train_trans = pd.DataFrame(x, columns = dataset_onehot.get_feature_names_out(colunas))
df_train_trans.head()

,onehotencoder__account.Contract_Month-to-month,onehotencoder__account.Contract_One year,onehotencoder__account.Contract_Two year,onehotencoder__account.PaymentMethod_Bank transfer (automatic),onehotencoder__account.PaymentMethod_Credit card (automatic),onehotencoder__account.PaymentMethod_Electronic check,onehotencoder__account.PaymentMethod_Mailed check,remainder__customer.gender,remainder__customer.SeniorCitizen,remainder__customer.Partner,remainder__customer.Dependents,remainder__customer.tenure,remainder__account.Charges.Total,remainder__daily_bill
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,9.0,593.30,2.20
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,9.0,542.40,2.01
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,280.85,2.34
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,13.0,1237.85,3.17
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,267.40,2.97


In [113]:
# Renaming columns for better readability
df_train_trans.columns = [c.replace('onehotencoder__', '') for c in df_train_trans.columns]
df_train_trans.columns = [c.replace('(automatic)', '') for c in df_train_trans.columns]
df_train_trans.columns = [c.replace('remainder__', '') for c in df_train_trans.columns]
df_x_train = df_train_trans.copy()
df_x_train.head()

,account.Contract_Month-to-month,account.Contract_One year,account.Contract_Two year,account.PaymentMethod_Bank transfer,account.PaymentMethod_Credit card,account.PaymentMethod_Electronic check,account.PaymentMethod_Mailed check,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,account.Charges.Total,daily_bill
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,9.0,593.30,2.20
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,9.0,542.40,2.01
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,280.85,2.34
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,13.0,1237.85,3.17
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,267.40,2.97


## Churn Exploratory Analysis

In [118]:
df_y_train = y.reset_index(drop=True).to_frame()
df_y_train['0'] = df_y_train['Churn']
df_y_train.drop('0', axis=1, inplace=True)
df_y_train.head()

y_churn = df_y_train['Churn'].value_counts()[1]
y_n_churn = df_y_train['Churn'].value_counts()[0]

px.bar(x = ['Churn', 'No Churn'],
       y = [y_churn, y_n_churn],
       title = 'Churn vs No Churn',
       labels = {'x': 'Customer Status', 'y': 'Number of Customers'})
